![piggy_bank](piggy_bank.jpg)

<br>

Project Overview:
In this project, I was tasked to work with an open source data base from a bank to clean, store, and design a PostgreSQL database for data collected during a recent marketing campaign promoting personal loans. The bank's objective was not only to organize the data but also to establish a flexible schema that could accommodate data from future campaigns. The data cleaning, transformation, and database creation process was performed using Python's Pandas library. Additionally, SQL scripts were prepared for table creation and data population.

## Project Goals

1. **Clean and preprocess the provided "bank_marketing.csv" dataset.**
   - Load the dataset into a Pandas DataFrame.
   - Address missing values, inconsistencies, and data type issues.
   - Format and transform the data for database storage.

2. **Design a PostgreSQL database schema to store campaign data.**
   - Create a relational database schema with appropriate tables.
   - Ensure scalability for future marketing campaigns by designing a flexible structure.

3. **Develop SQL scripts for creating the necessary database tables.**
   - Write SQL scripts for table creation in PostgreSQL.

4. **Prepare SQL scripts for populating the tables with data from the CSV file.**
   - Create SQL scripts for importing data from "bank_marketing.csv" into the database.
   - Ensure data integrity and consistency during the import process.

5. **Ensure that the database structure is adaptable for future marketing campaigns.**
   - Design the database schema to be easily extendable for upcoming campaigns.

## Skills and Tools Utilized

- Data Cleaning and Transformation with Pandas
- Relational Database Design (PostgreSQL)
- SQL Scripting
- Database Schema Design
- Data Import and Export

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('bank_marketing.csv')
df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Creating Dataframes for clients, campaigns and economics

In [3]:
# client 
subset_df_client = df[["client_id", "age", "job", "marital", "education", "credit_default", "housing", "loan"]]
# campaign
subset_df_campaign = df[['client_id', 'campaign', 'month', 'day', 'duration', 'pdays', 'previous', 'poutcome', 'y']]
# economics
subset_df_economics = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]

## Renaming columns

In [8]:
subset_df_client = subset_df_client.rename(columns={'client_id':'id'}) 

# renaming columns : "duration" to "contact_duration", "previous" to "previous_campaign_contacts", "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts"
subset_df_campaign = subset_df_campaign.rename(columns={'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign':'number_contacts'})

# renaming columns: "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed"
subset_df_economics = subset_df_economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed':'number_employed'})


In [9]:
from IPython.display import display

# Display the first 5 rows of the first DataFrame
display(subset_df_economics.head())

# Display the first 5 rows of the second DataFrame
display(subset_df_client.head())

# Display the first 5 rows of the third DataFrame
display(subset_df_campaign.head())

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,nonexistent,no
1,1,1,may,19,149,999,0,nonexistent,no
2,2,1,may,23,226,999,0,nonexistent,no
3,3,1,may,27,151,999,0,nonexistent,no
4,4,1,may,3,307,999,0,nonexistent,no


## Cleaning data

In [ ]:
# replacing '.' with '_' and unknown values to null numpy values
subset_df_client['education'] = subset_df_client['education'].str.replace('.', '_').replace('unknown', np.NaN)

# removing '.' in job column 
subset_df_client['job'] = subset_df_client['job'].str.replace('.', '')

### Converting "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome".

In [ ]:
subset_df_campaign['previous_outcome'] = subset_df_campaign['previous_outcome'].str.replace('success', '1').str.replace('failure', '0').replace('nonexistent', np.NaN)

# campaign_ouput column
subset_df_campaign['campaign_outcome'] = subset_df_campaign['campaign_outcome'].str.replace('success', '1').str.replace('failure', '0')

### Agregating a date column using existing columns.

In [10]:
# adding a campaign_column to dataframe 
subset_df_campaign['camapaign_id'] = '1'

# capitilizing month
subset_df_campaign['month'] = subset_df_campaign['month'].str.capitalize()

# adding year column
subset_df_campaign['year'] = '2022'

# changing the day column as string
subset_df_campaign['day'] = subset_df_campaign['day'].astype(str)

# creating a last_contact_date column
subset_df_campaign['last_contact_date'] = subset_df_campaign['year'] + '-' + subset_df_campaign['month'] + '-' + subset_df_campaign['day']

# format date
subset_df_campaign['last_contact_date'] = pd.to_datetime(subset_df_campaign['last_contact_date'], format="%Y-%b-%d")

# droping redundancy
column_to_remove = ['month', 'day', 'year']
subset_df_campaign.drop(columns=column_to_remove, inplace=True)

subset_df_campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,camapaign_id,last_contact_date
0,0,1,261,999,0,nonexistent,no,1,2022-05-13
1,1,1,149,999,0,nonexistent,no,1,2022-05-19
2,2,1,226,999,0,nonexistent,no,1,2022-05-23
3,3,1,151,999,0,nonexistent,no,1,2022-05-27
4,4,1,307,999,0,nonexistent,no,1,2022-05-03


## Saving dataframes as csv

In [ ]:
# converting datagrames into csv files
subset_df_client.to_csv('client.csv', index=False)
subset_df_campaign.to_csv('campaign.csv', index=False)
subset_df_economics.to_csv('economics.csv', index=False)

## Designing the database

In [12]:
# converting datagrames into csv files
subset_df_client.to_csv('client.csv', index=False)
subset_df_campaign.to_csv('campaign.csv', index=False)
subset_df_economics.to_csv('economics.csv', index=False)

In [13]:
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);

\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""
campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id) INTEGER,
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER, 
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);

\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""
economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);

\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""
